In [1]:
import torch
import torch.autograd as autograd         # computation graph
from torch import Tensor                  # tensor node in the computation graph
import torch.nn as nn                     # neural networks
import torch.optim as optim               # optimizers e.g. gradient descent, ADAM, etc.

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.ticker
from torch.nn.parameter import Parameter
import matplotlib as mpl

import numpy as np
import time
#from pyDOE import lhs         #Latin Hypercube Sampling
import scipy.io
from scipy.io import savemat

from smt.sampling_methods import LHS

#Set default dtype to float32
torch.set_default_dtype(torch.float)

#PyTorch random number generator
torch.manual_seed(1234)

# Random number generators in other libraries
np.random.seed(1234)

# Device configuration
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')

print(device)

if device == 'cuda': 
    print(torch.cuda.get_device_name())

cuda:1


In [2]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [3]:
# %cd '/content/gdrive/MyDrive/Virginia Tech /Fall 2022/Codes from GPU/MURI Aug17 Thin Plate/Rowdy'

In [4]:
# !pip install smt

In [5]:
lr_tune = np.array([0.05,0.1,0.25,0.5,1]).reshape(-1,1)
n_value = np.array([1.0,3.0,5.0,8.0,10.0]).reshape(-1,1)
r_value = np.array([2,6,8]).reshape(-1,1)

LR_tune,N_value,R_value = np.meshgrid(lr_tune,n_value,r_value)

LR_tune = LR_tune.flatten('F').reshape(-1,1)
N_value = N_value.flatten('F').reshape(-1,1)
R_value = R_value.flatten('F').reshape(-1,1)


lrnr_tune = np.hstack((LR_tune,N_value,R_value))

In [6]:
#Material Properties This link - https://www.mathworks.com/help/pde/ug/nonlinear-heat-transfer-in-a-thin-plate.html#heatTransferThinPlateExample-1
k = 400
rho = 8960
cp = 386
t_z = 0.01
stef_bolt = 5.670373e-8
hc = 1
Ta = 300
emiss = 0.5


In [7]:
loss_thresh = 10000

x = np.linspace(0,1,100).reshape(-1,1)
y = np.linspace(0,1,100).reshape(-1,1)
t = np.linspace(0,1,100).reshape(-1,1) #t is actually from 0 to 3000, let us scale it to 0 to 1

X,Y,T = np.meshgrid(x,y,t)

X = X.flatten('F').reshape(-1,1)
Y = Y.flatten('F').reshape(-1,1)
T = T.flatten('F').reshape(-1,1)
  
xyt = np.hstack((X,Y,T))

initial_pts = np.logical_and(T==0,Y!=0).reshape(-1,)

DBC_pts = (Y == 0).reshape(-1,)


NBC_pts_x0 = (X == 0).reshape(-1,)
NBC_pts_x1 = (X == 1).reshape(-1,)

NBC_pts_y0 = (Y == 0).reshape(-1,)
NBC_pts_y1 = (Y == 1).reshape(-1,)

xyt_initial = xyt[initial_pts,:]
xyt_DBC = xyt[DBC_pts,:]

xyt_NBC_x0 = xyt[NBC_pts_x0,:]
xyt_NBC_x1 = xyt[NBC_pts_x1,:]

#xyt_NBC_y0 = xyt[NBC_pts_y0,:]
xyt_NBC_y1 = xyt[NBC_pts_y1,:]

u_initial = 300*np.ones((np.shape(xyt_initial)[0],1))
u_DBC = 1000*np.ones((np.shape(xyt_DBC)[0],1))

xyt_I_DBC = np.vstack((xyt_initial,xyt_DBC))
#xyt_NBC = np.vstack((xyt_NBC_1,xyt_NBC_2,xyt_NBC_3,xyt_NBC_4))
xyt_NBC_x = np.vstack((xyt_NBC_x0,xyt_NBC_x1))
#xyt_NBC_y = np.vstack((xyt_NBC_y0,xyt_NBC_y1))
xyt_NBC_y = np.vstack((xyt_NBC_y1))

u_I_DBC = np.vstack((u_initial,u_DBC))



lb_xyt = xyt[0]
ub_xyt = xyt[-1]

In [8]:
fea_data = scipy.io.loadmat('./../3D_HTTP_FEA.mat')
xy = fea_data['xy']
t = fea_data['t']/3000
xyt = np.zeros((497*101,3))
u_true = np.ones((497*101,1))


for i in range(101):
    t_temp = t[0,i]*np.ones((497,1))
    xyt[497*i:497*(i+1)] = np.hstack((xy,t_temp))
    u_true[497*i:497*(i+1)] = fea_data['u'][:,i].reshape(-1,1)
    #print(i)
#print(xyt)

xyt_test_tensor = torch.from_numpy(xyt).float().to(device)
u_true_norm = np.linalg.norm(u_true,2)

In [9]:
def trainingdata(N_D,N_N,N_f,seed):
    '''Boundary Conditions''' 
    
    np.random.seed(seed)
    
    #choose random N_u points for training
    idx = np.random.choice(xyt_I_DBC.shape[0], N_D, replace=False) 
    xyt_D = xyt_I_DBC[idx,:] #choose indices from  set 'idx' (x,t)
    u_D = u_I_DBC[idx].reshape(-1,1)      #choose corresponding u

    idx = np.random.choice(xyt_NBC_x.shape[0], N_D, replace=False) 
    xyt_Nx = xyt_NBC_x[idx,:] #choose indices from  set 'idx' (x,t)

    idx = np.random.choice(xyt_NBC_y.shape[0], N_D, replace=False) 
    xyt_Ny = xyt_NBC_y[idx,:] #choose indices from  set 'idx' (x,t)

    '''Collocation Points'''
    # Latin Hypercube sampling for collocation points 
    # N_f sets of tuples(x,t)
    x01 = np.array([[0.0,1.0],[0.0,1.0],[0.0,1.0]])
    sampling = LHS(xlimits=x01,random_state =seed)
    samples = sampling(N_f)
    
    xyt_coll = lb_xyt + (ub_xyt - lb_xyt)*samples
    xyt_coll = np.vstack((xyt_coll, xyt_D,xyt_Nx,xyt_Ny)) # append training points to collocation points 

    return xyt_coll, xyt_D, u_D, xyt_Nx,xyt_Ny

In [10]:
class Sequentialmodel(nn.Module):
    
    def __init__(self,layers,n_val,rowdy_terms):
        super().__init__() #call __init__ from parent class 
              
        'activation function'
        self.activation = nn.Tanh()
     
        'loss function'
        self.loss_function = nn.MSELoss(reduction ='mean')
        
        'Initialise neural network as a list using nn.Modulelist'  
        self.linears = nn.ModuleList([nn.Linear(layers[i], layers[i+1]) for i in range(len(layers)-1)])
        self.iter = 0
        
        # std = gain * sqrt(2/(input_dim+output_dim))
        for i in range(len(layers)-1):
            nn.init.xavier_normal_(self.linears[i].weight.data, gain=1.0)
            # set biases to zero
            nn.init.zeros_(self.linears[i].bias.data)   

        
        self.omega1 = Parameter(torch.ones((len(layers)-2,1))) 
        self.omega1.requiresGrad = True
        
        
        self.alpha = Parameter(torch.zeros(rowdy_terms,len(layers)-2))
        self.alpha.requiresGrad = True
        
        
        self.omega = Parameter((1/n_val)*torch.ones(rowdy_terms,len(layers)-2))
        self.omega.requiresGrad = True
        
        self.n = torch.tensor(n_val)
            
    'foward pass'
    def forward(self,xyt):
        if torch.is_tensor(xyt) != True:         
            xyt = torch.from_numpy(xyt)                
        
        ubxyt = torch.from_numpy(ub_xyt).float().to(device)
        lbxyt = torch.from_numpy(lb_xyt).float().to(device)
    
                      
        #preprocessing input 
        xyt = (xyt - lbxyt)/(ubxyt - lbxyt)
        
        #convert to float
        a = xyt.float()
        
        for i in range(len(layers)-2):
            z = self.linears[i](a)
            a = self.activation(self.omega1[i,0]*z)
            for j in range(rowdy_terms):
                a = a + self.alpha[j,i]*self.n*torch.sin((j+1)*self.n*self.omega[j,i]*z)
                
        a = self.linears[-1](a) 
         
        return a
                        
    def loss_D(self,xyt_D,u_D):
                
        loss_bc = self.loss_function(self.forward(xyt_D), u_D)
                
        return loss_bc
    
    def loss_N(self,xyt_Nx,xyt_Ny,N_hat):
        
        g1 = xyt_Nx.clone()             
        g1.requires_grad = True
        u1 = self.forward(g1)
        
        u1_x_y_t = autograd.grad(u1,g1,torch.ones([xyt_Nx.shape[0], 1]).to(device), retain_graph=True, create_graph=True,allow_unused = True)[0]
        
        du1_dx = u1_x_y_t[:,[0]]
        
        g2 = xyt_Ny.clone()             
        g2.requires_grad = True
        u2 = self.forward(g2)
        
        u2_x_y_t = autograd.grad(u2,g2,torch.ones([xyt_Ny.shape[0], 1]).to(device), retain_graph=True, create_graph=True,allow_unused = True)[0]
        
        du2_dy = u2_x_y_t[:,[1]]
               
        loss_N1 = self.loss_function(du1_dx,N_hat)
        loss_N2 = self.loss_function(du2_dy,N_hat)
        
        #return loss_N1+loss_N2       
        return loss_N1 + loss_N2
    
    def loss_PDE(self, xyt_coll, f_hat):
        
        g = xyt_coll.clone()             
        g.requires_grad = True
        u = self.forward(g) 
        
        u_x_y_t = autograd.grad(u,g,torch.ones([xyt_coll.shape[0], 1]).to(device), retain_graph=True, create_graph=True,allow_unused = True)[0]
        
        u_xx_yy_tt = autograd.grad(u_x_y_t,g,torch.ones(xyt_coll.shape).to(device), create_graph=True,allow_unused = True)[0]

        du_dt = u_x_y_t[:,[2]]
        
        d2u_dx2 = u_xx_yy_tt[:,[0]]
        d2u_dy2 = u_xx_yy_tt[:,[1]]    
        

        f = rho*cp*t_z*du_dt/3000 - k*t_z*(d2u_dx2+d2u_dy2) + 2*hc*(u-Ta) + 2*emiss*stef_bolt*(torch.pow(u,4)-Ta**4) 
        
        loss_f = self.loss_function(f,f_hat)
                
        return loss_f
    
    def loss(self,xyt_D,u_D,xyt_Nx,xyt_Ny,N_hat,xyt_coll,f_hat):

        loss_D = self.loss_D(xyt_D,u_D)
        loss_N = self.loss_N(xyt_Nx,xyt_Ny,N_hat)
        loss_f = self.loss_PDE(xyt_coll,f_hat)
        
        loss_val = loss_D + loss_N + loss_f
        
        #print(self.iter,"loss_D:",loss_D.cpu().detach().numpy(),"loss_N:",loss_N.cpu().detach().numpy(),"loss_f:",loss_f.cpu().detach().numpy())
        
        return loss_val
          
    'test neural network'
    def test(self):
        u_pred = self.forward(xyt_test_tensor)
        u_pred = u_pred.cpu().detach().numpy()
   
        return u_pred

    def test_loss(self):
        u_pred = self.test()
               
        test_mse = np.mean(np.square(u_pred.reshape(-1,1) - u_true.reshape(-1,1)))
        test_re = np.linalg.norm(u_pred.reshape(-1,1) - u_true.reshape(-1,1),2)/u_true_norm
        
        return test_mse, test_re 

In [11]:
def train_step(xyt_D,u_D,xyt_Nx,xyt_Ny,N_hat,xyt_coll,f_hat,seed):    
    def closure():
        optimizer.zero_grad()
        loss = PINN.loss(xyt_D,u_D,xyt_Nx,xyt_Ny,N_hat,xyt_coll,f_hat)
        loss.backward()
        
        return loss

    optimizer.step(closure)

In [12]:
def data_update(loss_np):
    train_loss.append(loss_np)
    alpha_val.append(PINN.alpha.cpu().detach().numpy())
    omega_val.append(PINN.omega.cpu().detach().numpy())
    
    test_mse, test_re = PINN.test_loss()
    test_mse_loss.append(test_mse)
    test_re_loss.append(test_re)

In [13]:
def train_model(max_iter,rep): 
    print(rep) 
    torch.manual_seed(rep*11)
    start_time = time.time() 
    thresh_flag = 0
    
    xyt_coll_np_array, xyt_D_np_array, u_D_np_array,xyt_Nx_np_array,xyt_Ny_np_array = trainingdata(N_D,N_N,N_f,(reps)*22)

    xyt_coll = torch.from_numpy(xyt_coll_np_array).float().to(device)
    xyt_D = torch.from_numpy(xyt_D_np_array).float().to(device)
    u_D = torch.from_numpy(u_D_np_array).float().to(device)
    xyt_Nx = torch.from_numpy(xyt_Nx_np_array).float().to(device)
    xyt_Ny = torch.from_numpy(xyt_Ny_np_array).float().to(device)

    N_hat = torch.zeros(xyt_Nx.shape[0],1).to(device)    
    f_hat = torch.zeros(xyt_coll.shape[0],1).to(device)

    nan_flag = 0
    
    for i in range(max_iter):
        train_step(xyt_D,u_D,xyt_Nx,xyt_Ny,N_hat,xyt_coll,f_hat,i)

        loss_np = PINN.loss(xyt_D,u_D,xyt_Nx,xyt_Ny,N_hat,xyt_coll,f_hat).cpu().detach().numpy()
        
        if(thresh_flag == 0):
            if(loss_np < loss_thresh):
                time_threshold[rep] = time.time() - start_time
                epoch_threshold[rep] = i+1            
                thresh_flag = 1       
        data_update(loss_np)
        print(i,"Train Loss",train_loss[-1],"Test MSE",test_mse_loss[-1],"Test RE",test_re_loss[-1])

        if(np.isnan(loss_np)):
            nan_flag =1
            print("NAN BREAK!")
            break
    
    elapsed_time[rep] = time.time() - start_time  
    print('Training time: %.2f' % (elapsed_time[rep]))
    
    return nan_flag

In [14]:
nan_tune = []
for tune_reps in range(50,75):
    label = "3D_HTTP_rowdy_tune"+str(tune_reps)
    
    max_reps = 10 #10
    max_iter = 100 #1000

    train_loss_full = []
    test_mse_full = []
    test_re_full = []
    alpha_full = []
    omega_full = []
    elapsed_time= np.zeros((max_reps,1))
    
    time_threshold = np.empty((max_reps,1))
    time_threshold[:] = np.nan
    epoch_threshold = max_iter*np.ones((max_reps,1))
    
    n_val = lrnr_tune[tune_reps,1]
    rowdy_terms = int(lrnr_tune[tune_reps,2])

    for reps in range(max_reps):
        print(label)
        print(reps)

        train_loss = []
        test_mse_loss = []
        test_re_loss = []

        alpha_val = []    
        omega_val = []

        torch.manual_seed(reps*36)
        N_D = 5000 #Total number of data points for 'y'
        N_N = 3500
        N_f = 10000#Total number of collocation points 

        layers = np.array([3,50,50,50,50,50,50,50,50,50,1]) #9 hidden layers


        PINN = Sequentialmodel(layers,n_val,rowdy_terms)

        PINN.to(device)

        'Neural Network Summary'
        print(PINN)

        params = list(PINN.parameters())

        optimizer = torch.optim.LBFGS(PINN.parameters(), lr=lrnr_tune[tune_reps,0], 
                                  max_iter = 10, 
                                  max_eval = 15, 
                                  tolerance_grad = 1e-6, 
                                  tolerance_change = 1e-6, 
                                  history_size = 100, 
                                  line_search_fn = 'strong_wolfe')


        nan_flag = train_model(max_iter,reps)

        torch.save(PINN.state_dict(),label+'_'+str(reps)+'.pt')
        train_loss_full.append(train_loss)
        test_mse_full.append(test_mse_loss)
        test_re_full.append(test_re_loss)
        #elapsed_time[reps] = time.time() - start_time
        alpha_full.append(alpha_val)
        omega_full.append(omega_val)
        
        if(nan_flag == 1):
            nan_tune.append(tune_reps)
            break


        #print('Training time: %.2f' % (elapsed_time[reps]))

    mdic = {"train_loss": train_loss_full,"test_mse_loss": test_mse_full,"test_re_loss": test_re_full,"Time": elapsed_time,"alpha": alpha_full,"omega": omega_full,  "label": label}
    savemat(label+'.mat', mdic)

3D_HTTP_rowdy_tune50
0
Sequentialmodel(
  (activation): Tanh()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=3, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): Linear(in_features=50, out_features=50, bias=True)
    (4): Linear(in_features=50, out_features=50, bias=True)
    (5): Linear(in_features=50, out_features=50, bias=True)
    (6): Linear(in_features=50, out_features=50, bias=True)
    (7): Linear(in_features=50, out_features=50, bias=True)
    (8): Linear(in_features=50, out_features=50, bias=True)
    (9): Linear(in_features=50, out_features=1, bias=True)
  )
)
0
0 Train Loss 1595508.8 Test MSE 316765.6657505402 Test RE 0.9977193918737586
1 Train Loss 1411065.2 Test MSE 315238.9885602403 Test RE 0.9953121941736155
2 Train Loss 1169156.8 Test MSE 315497.1402241665 Test RE 0.9957196452298819
3 Train Loss 970708.75 Test MSE 314516.1719727

96 Train Loss 179153.83 Test MSE 47340.52592694249 Test RE 0.3857055618391762
97 Train Loss 178763.2 Test MSE 47502.75662830788 Test RE 0.38636588157935225
98 Train Loss 178564.48 Test MSE 47464.43053685002 Test RE 0.3862099866040895
99 Train Loss 178450.62 Test MSE 47417.42110585132 Test RE 0.3860186853590718
Training time: 558.77
3D_HTTP_rowdy_tune50
1
Sequentialmodel(
  (activation): Tanh()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=3, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): Linear(in_features=50, out_features=50, bias=True)
    (4): Linear(in_features=50, out_features=50, bias=True)
    (5): Linear(in_features=50, out_features=50, bias=True)
    (6): Linear(in_features=50, out_features=50, bias=True)
    (7): Linear(in_features=50, out_features=50, bias=True)
    (8): Linear(in_features=50, out_features=50, bias=True)
    (9): L

91 Train Loss 57695.28 Test MSE 9850.59362589757 Test RE 0.17594242105400554
92 Train Loss 57345.0 Test MSE 9769.522758821462 Test RE 0.17521691787532256
93 Train Loss 56886.547 Test MSE 9659.262569971446 Test RE 0.17422535097763966
94 Train Loss 56434.18 Test MSE 9635.313571725823 Test RE 0.17400923136577126
95 Train Loss 55498.34 Test MSE 9300.049663314932 Test RE 0.17095507429683865
96 Train Loss 54981.64 Test MSE 9109.788862525258 Test RE 0.16919733487311733
97 Train Loss 54488.28 Test MSE 8930.043325480887 Test RE 0.16751979974440578
98 Train Loss 54173.254 Test MSE 8829.966172823524 Test RE 0.16657847509364077
99 Train Loss 54017.777 Test MSE 8786.499555093038 Test RE 0.16616796754178909
Training time: 552.44
3D_HTTP_rowdy_tune50
2
Sequentialmodel(
  (activation): Tanh()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=3, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_fea

87 Train Loss 130374.445 Test MSE 31228.262382757468 Test RE 0.31326600743644145
88 Train Loss 130199.77 Test MSE 31225.374003823545 Test RE 0.3132515197289766
89 Train Loss 130092.56 Test MSE 31107.890715480004 Test RE 0.3126616709099748
90 Train Loss 129720.23 Test MSE 30586.705619903656 Test RE 0.31003142312941284
91 Train Loss 128882.586 Test MSE 29786.26131566521 Test RE 0.3059478177543068
92 Train Loss 127803.06 Test MSE 29346.97091312882 Test RE 0.30368336498770815
93 Train Loss 126222.13 Test MSE 28946.39719087397 Test RE 0.30160366921316945
94 Train Loss 124206.11 Test MSE 28776.382004071507 Test RE 0.3007166379314383
95 Train Loss 123589.37 Test MSE 28723.189110078296 Test RE 0.30043857331593476
96 Train Loss 123253.375 Test MSE 28595.841479361698 Test RE 0.299771818619616
97 Train Loss 123006.16 Test MSE 28649.355896856585 Test RE 0.3000521848392178
98 Train Loss 122425.375 Test MSE 28825.76862016752 Test RE 0.30097457533565714
99 Train Loss 121987.52 Test MSE 28649.79227785

83 Train Loss 251533.16 Test MSE 90494.43345448712 Test RE 0.5332738587209613
84 Train Loss 247850.61 Test MSE 88516.90318775643 Test RE 0.5274149887291512
85 Train Loss 246547.75 Test MSE 87232.72309641479 Test RE 0.5235752118388468
86 Train Loss 245697.11 Test MSE 86609.21692041837 Test RE 0.5217006984687911
87 Train Loss 245136.05 Test MSE 86427.17050581405 Test RE 0.5211521211662604
88 Train Loss 244294.75 Test MSE 85958.16814602846 Test RE 0.5197361655810729
89 Train Loss 243093.05 Test MSE 84661.12756636231 Test RE 0.5158000577009894
90 Train Loss 242340.48 Test MSE 84063.26648559723 Test RE 0.513975589113746
91 Train Loss 241764.56 Test MSE 83772.84859826678 Test RE 0.5130869914245051
92 Train Loss 240871.6 Test MSE 83588.6063389721 Test RE 0.5125224628109015
93 Train Loss 237843.8 Test MSE 81048.31075333708 Test RE 0.5046744820297486
94 Train Loss 236728.23 Test MSE 80022.78168556088 Test RE 0.5014714171356449
95 Train Loss 236390.95 Test MSE 79766.54273314559 Test RE 0.5006678

...

9 Train Loss 383462.7 Test MSE 163281.2790484057 Test RE 0.7163207950492974
10 Train Loss 333966.28 Test MSE 137700.03083763478 Test RE 0.657818951173706
11 Train Loss 281875.88 Test MSE 102327.78268569056 Test RE 0.5670693103465638
12 Train Loss 246017.0 Test MSE 87198.65219605247 Test RE 0.5234729542202824
13 Train Loss 241345.66 Test MSE 83910.27107148079 Test RE 0.5135076574987333
14 Train Loss 234929.94 Test MSE 78461.45655740998 Test RE 0.49655521258681695
15 Train Loss 230258.89 Test MSE 75703.80160959042 Test RE 0.4877510429208591
16 Train Loss 226941.33 Test MSE 74073.72892528282 Test RE 0.48247127987892846
17 Train Loss 223537.83 Test MSE 71657.49453062513 Test RE 0.47453709993341736
18 Train Loss 221323.66 Test MSE 69098.27032009812 Test RE 0.4659860872009415
19 Train Loss 218203.89 Test MSE 67013.5327673284 Test RE 0.4589027062923645
20 Train Loss 215456.03 Test MSE 66014.0124773302 Test RE 0.4554675361270994
21 Train Loss 213097.83 Test MSE 63545.23223987705 Test RE 0.4468

0 Train Loss 1617609.9 Test MSE 317981.3427410267 Test RE 0.9996320723678312
1 Train Loss 1518135.8 Test MSE 318030.174913777 Test RE 0.9997088258208435
2 Train Loss 1229069.9 Test MSE 317256.3469270137 Test RE 0.99849184409532
3 Train Loss 801601.5 Test MSE 302468.71618236875 Test RE 0.9749438267060906
4 Train Loss 589917.6 Test MSE 303816.0767387999 Test RE 0.9771128795076346
5 Train Loss 541746.5 Test MSE 295831.1948945056 Test RE 0.9641871651609275
6 Train Loss 500541.66 Test MSE 268558.1668466454 Test RE 0.9186679053603536
7 Train Loss 428180.9 Test MSE 203311.72186797124 Test RE 0.7993198007262698
8 Train Loss 384426.0 Test MSE 178691.08289509357 Test RE 0.7493605096105758
9 Train Loss 351122.4 Test MSE 154566.45403896706 Test RE 0.696942556474981
10 Train Loss 326541.0 Test MSE 133323.18842801976 Test RE 0.647280029327869
11 Train Loss 290078.5 Test MSE 109662.9986677391 Test RE 0.5870423336025019
12 Train Loss 252737.25 Test MSE 89087.61879719538 Test RE 0.5291125195426221
13 T

0 Train Loss 1631095.5 Test MSE 318075.2842423779 Test RE 0.9997797225446314
1 Train Loss 1408379.6 Test MSE 313330.1529161842 Test RE 0.9922942105910023
2 Train Loss nan Test MSE nan Test RE nan
NAN BREAK!
Training time: 17.30
3D_HTTP_rowdy_tune72
0
Sequentialmodel(
  (activation): Tanh()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=3, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): Linear(in_features=50, out_features=50, bias=True)
    (4): Linear(in_features=50, out_features=50, bias=True)
    (5): Linear(in_features=50, out_features=50, bias=True)
    (6): Linear(in_features=50, out_features=50, bias=True)
    (7): Linear(in_features=50, out_features=50, bias=True)
    (8): Linear(in_features=50, out_features=50, bias=True)
    (9): Linear(in_features=50, out_features=1, bias=True)
  )
)
0
0 Train Loss 1617564.2 Test MSE 318167.1705259385

82 Train Loss 200749.69 Test MSE 61566.59596966833 Test RE 0.43985743082309325
83 Train Loss 200710.31 Test MSE 61526.036740875075 Test RE 0.43971252093248625
84 Train Loss 200389.39 Test MSE 61489.68906997524 Test RE 0.43958261749614774
85 Train Loss 200109.8 Test MSE 61217.82305048781 Test RE 0.4386097717507112
86 Train Loss 199853.64 Test MSE 61104.917776933784 Test RE 0.43820511663407136
87 Train Loss 199655.14 Test MSE 61145.76325773753 Test RE 0.4383515509084397
88 Train Loss 199520.42 Test MSE 61103.89519202438 Test RE 0.4382014499585356
89 Train Loss 199262.4 Test MSE 60667.413349368435 Test RE 0.43663354852403635
90 Train Loss 198570.33 Test MSE 60001.45730554731 Test RE 0.4342304368209839
91 Train Loss 198282.33 Test MSE 59665.85595906799 Test RE 0.43301436082599964
92 Train Loss 197285.78 Test MSE 58616.70725059198 Test RE 0.4291904714285859
93 Train Loss 195974.12 Test MSE 58148.05942226763 Test RE 0.42747131284549666
94 Train Loss 193610.17 Test MSE 56522.4148872659 Test R

18 Train Loss 230165.9 Test MSE 75479.22758385557 Test RE 0.4870270530552697
19 Train Loss 228784.36 Test MSE 73477.96157411997 Test RE 0.48052712950035525
20 Train Loss 227715.05 Test MSE 73437.31516266445 Test RE 0.48039420253233284
21 Train Loss 226475.0 Test MSE 72823.39097309121 Test RE 0.47838197895417917
22 Train Loss 225888.45 Test MSE 72138.3988946933 Test RE 0.47612678263351554
23 Train Loss 224634.92 Test MSE 72069.76073223406 Test RE 0.47590021644025043
24 Train Loss 223632.28 Test MSE 69566.91421832317 Test RE 0.46756364120154603
25 Train Loss 220688.6 Test MSE 66554.74953666031 Test RE 0.4573291551243555
26 Train Loss 216837.58 Test MSE 64853.7610971391 Test RE 0.451447181450306
27 Train Loss 215343.17 Test MSE 65874.08453324455 Test RE 0.4549845595406078
28 Train Loss 212208.98 Test MSE 64758.513314672586 Test RE 0.451115549625048
29 Train Loss 209277.61 Test MSE 61280.633604779854 Test RE 0.4388347247024052
30 Train Loss 208169.44 Test MSE 61198.09927548765 Test RE 0.43

In [15]:
import scipy.io as sio
import numpy as np

In [18]:
for tune_reps in range(75):
    label = "3D_HTTP_rowdy_tune"+str(tune_reps)+".mat"
    data = sio.loadmat(label)
    re = np.array(data["test_re_loss"])
    print(tune_reps," ",np.mean(re[:,-1]))

0   0.2837226813890388
1   [[0.99975842        nan]]
2   [[0.98188853        nan]]
3   0.37183695061628896
4   [[0.99998504        nan]]
5   0.28092429464070273
6   0.405797353842722
7   [[nan]]
8   [[nan]]
9   [[0.99160903 0.97554508 0.98297957 0.97857823 0.97121284 0.9641238
  0.95019369 0.91586793 0.88957541 0.81827802 0.77357601 0.61446989
         nan]]
10   0.265083334916372
11   nan
12   [[1.00013967 0.97433933 0.96627759 0.93622022 0.87815627 0.77446999
  0.67910451 0.65556652        nan]]
13   nan
14   nan
15   [[0.99708224 0.9705087  0.85000192 0.58794592        nan]]
16   nan
17   [[nan]]
18   [[0.98507226 0.97579785 0.96892242 0.96166809 0.94218618 0.90458512
  0.86500248 0.70992727 0.6700939  0.55405574        nan]]
19   nan
20   [[0.99046477        nan]]
21   nan
22   nan
23   nan
24   nan
25   [[0.99976937        nan]]
26   0.3587737131339293
27   0.4192743970624761
28   0.4602081728675717
29   0.3970014287688981
30   0.25969635295003834
31   [[1.00000762 0.99541721     

In [17]:
lrnr_tune[30]

array([0.1, 1. , 6. ])